# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [40]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Write your code here

## Getting Started
I printed the info, the different types of crops (the dependent variable) and the head of the dataset to see what I was working with.

In [41]:
print(crops.info())
print(crops['crop'].unique())
crops.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None
['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


## Finding the Best Feature
I started by creating a list of all the features. I then initialized an empty dictionary to store my evaluation score for each feature's model. I next created a for loop and looped through all the features to create, fit, make predictions with, and score a logistic regression model for each feature. Within the loop I appended the feature's name and the R-squared value of the feature's logistic regression model to my feature_r_squared dictionary. Upon viewing the dictionary after the for loop was run, it appeared that K, the potassium content ratio in the soil, is the best predictor of which crop was planted, with an R-squared value of 0.275. To increase confidence in my results, I performed a K-fold cross-validation on each model, using 5 folds. Just as with my first analysis, K produced the model with the highest mean R-squared across all features.

In [42]:
#Generating list of features
features = list(crops.drop(columns=['crop']).columns)
#Initializing an empty dictionary to store r-squared results in for each feature
feature_r_squared= {}
#Creating a for loop to loop through each feature, creating, running, and scoring the predictions of a logistic regression model for each
for feature in features:
    X = crops[[feature]].values
    y= crops[['crop']].values
    log_reg= LogisticRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=182)
    log_reg.fit(X_train, y_train)
    y_pred = log_reg.predict(X_test)
    r_squared_test = log_reg.score(X_test, y_test)
    feature_r_squared[feature] = r_squared_test
    
print(feature_r_squared)    

{'N': 0.1590909090909091, 'P': 0.2, 'K': 0.275, 'ph': 0.07045454545454545}


In [43]:
from sklearn.model_selection import cross_val_score, KFold
feature_r_squared_cross= {}
#Creating a for loop that loops through each feature and perform k-fold cross validation on its logistic regression model
for feature in features:
    X = crops[[feature]].values
    y= crops[['crop']].values
    log_reg= LogisticRegression()
    kf= KFold(n_splits = 5, shuffle=True, random_state=182)
    cv_results = cross_val_score(log_reg, X, y, cv=kf)
    feature_r_squared_cross[feature] = np.mean(cv_results)
    
print(feature_r_squared_cross)   

{'N': 0.14863636363636365, 'P': 0.20363636363636362, 'K': 0.3013636363636364, 'ph': 0.09227272727272727}


In [44]:
best_predictive_feature = {'K': feature_r_squared['K']}
print(best_predictive_feature)

{'K': 0.275}
